In [1]:
# Import packages
import matplotlib
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import matplotlib.pyplot as plt
import torch.utils.data as Data
import matplotlib.dates as mdates
from torch.autograd import Variable
import time
import datetime
import sklearn.metrics as sk
from dateutil.relativedelta import relativedelta
import calendar
import numpy as np
import joblib
import ast
import scipy as sp
import scipy.stats as st
import scipy.optimize as opt
from numpy.core.fromnumeric import trace
from numpy.linalg import inv
from scipy.optimize import root
import random
from scipy.stats import norm
from tkinter import _flatten
import sklearn.metrics as sk
from tqdm.notebook import trange, tqdm
from scipy.stats import wishart
from scipy.stats import invwishart
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.notebook import trange, tqdm

rng = np.random.default_rng()
device0 = torch.device('cuda' if torch.cuda.is_available else 'cpu')
np.random.seed(16)
torch.manual_seed(16)
torch.cuda.manual_seed(16) 
torch.set_default_dtype(torch.float32)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# load data
X_spt=joblib.load('X_cov')
Z=joblib.load('Data_nonGaussian')
# Z=joblib.load('Data_Gaussian')
X_spt=X_spt.float().cuda()
Z=Z.float().cuda()
T=1095
S=11025
Z=Z.T
Z.shape

In [ ]:
#LSTM & GRU
# calculate the scaling parameters using only the Training subset
Z_scaler=MinMaxScaler(feature_range=(0, 1))
Z_scaled_tmp=Z_scaler.fit_transform(torch.transpose(Z[:7025], 0,1).cpu())
Z_scaled_tmp=torch.tensor(Z_scaled_tmp).float().cuda().T
Z_scaled_tmp.shape

In [ ]:
X_LSTM=torch.repeat_interleave(X_spt,1095,dim=0)
X_LSTM=X_LSTM.reshape(11025,1095,11)

Z_max=np.mean(Z_scaler.data_max_)
Z_min=np.mean(Z_scaler.data_min_)
delta=Z_max-Z_min
def rescale(raw):
    return(raw*delta+Z_min)

class Simu_LSTM(nn.Module): 
    def __init__(self,dim_output=1, dim_input=11, dim_emb=64, dropout_input=0.8, dropout_emb=0.5, 
                 dropout_context=0.5,  l2=0.0001, batch_first=True):
        super(Simu_LSTM,self).__init__()
        self.batch_first = batch_first
        self.embedding = nn.Sequential(
            nn.Dropout(p=dropout_input),
            nn.Linear(dim_input, dim_emb, bias=False),
            nn.Dropout(p=dropout_emb)
        )
        init.xavier_normal_(self.embedding[1].weight)
        
        # self.rnn = nn.GRU(input_size=dim_emb, hidden_size=dim_emb, num_layers=5, batch_first=self.batch_first,bidirectional=True)
        
        self.rnn = nn.LSTM(input_size=dim_emb, hidden_size=dim_emb, num_layers=5, batch_first=self.batch_first,bidirectional=True)
        
        self.attention = nn.Linear(2*dim_emb,2*dim_emb,bias=True)
        init.xavier_normal_(self.attention.weight, gain=nn.init.calculate_gain('tanh'))
        self.attention.bias.data.zero_()
        
        
        self.output = nn.Sequential(
            nn.Dropout(p=dropout_context),
            nn.Linear(in_features=2*dim_emb, out_features=dim_output)
        )
        init.xavier_normal_(self.output[1].weight)
        self.output[1].bias.data.zero_()

    def forward(self,x):
        
        emb = self.embedding(x)
        
        h,_=self.rnn(emb)
        
        temp=self.output(F.relu(h))
        
        finaltemp=temp.reshape(temp.shape[0],temp.shape[1])
        finaltemp=rescale(finaltemp)
        
        return finaltemp,h
    
simulstm = Simu_LSTM().cuda()
print(simulstm)

In [ ]:
train_set = Data.TensorDataset(X_LSTM[:7025,:,:], Z[:7025,:])
train_loader=Data.DataLoader(dataset=train_set, batch_size=128,shuffle=False)

valid_set = Data.TensorDataset(X_LSTM[7025:9025,:,:] , Z[7025:9025,:])
valid_loader=Data.DataLoader(dataset=valid_set, batch_size=256,shuffle=False)

test_set = Data.TensorDataset(X_LSTM[9025:,:,:] , Z[9025:,:])
test_loader =Data.DataLoader(dataset=test_set, batch_size=256,shuffle=False)

optimizer = torch.optim.Adam(simulstm.parameters(),lr = 0.001)
loss_func = torch.nn.MSELoss()
    
def epoch(loader,train=False):
    if train:
        simulstm.train()
        mode = 'Train'
    else:
        simulstm.eval()
        mode = 'Eval'
    
    true_value = []
    outputs = []
    att =[]
    losses = 0

    for step ,(batch_x,batch_z) in enumerate(loader):
        input_var=Variable(batch_x)
        target_var=Variable(batch_z)
    
        prediction,att = simulstm(input_var)
        loss = loss_func(prediction,target_var)
        
        true_value.append(batch_z)
        outputs.append(prediction.data)
        losses=losses+loss.data
#         att.append(attention.data)
    
        # compute gradient and do update step
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return torch.cat(true_value,0),torch.cat(outputs, 0),losses/len(loader)

In [ ]:
# main
best_valid_epoch = 0
best_valid_loss = 1e10


train_losses = []
valid_losses = []
n_epochs=4000

time_start=time.time()

for ei in trange(n_epochs):
            
    train_y_true, train_y_pred, train_loss = epoch(train_loader,train=True)
    train_losses.append(train_loss)

#     Eval
    valid_y_true, valid_y_pred, valid_loss = epoch(valid_loader)
    valid_losses.append(valid_loss)

    valid_y_true = valid_y_true.cpu()
    valid_y_pred = valid_y_pred.cpu()


    is_best = valid_loss < best_valid_loss

    if is_best:
        best_valid_epoch = ei
        best_valid_loss = valid_loss


    # evaluate on the test set
        test_y_true,test_y_pred,  test_loss = epoch(test_loader)
        
        train_y_true_best = train_y_true.cpu()
        train_y_pred_best = train_y_pred.cpu()
        test_y_true = test_y_true.cpu()
        test_y_pred = test_y_pred.cpu()
        
        
        # with open('GRU_case.txt', 'w') as f:
        #     f.write('Best Validation Epoch: {}\n'.format(ei))
        #     f.write('Best Validation Loss: {}\n'.format(best_valid_loss))
        #     f.write('Train Loss: {}\n'.format(train_loss))
        #     f.write('Test Loss: {}\n'.format(test_loss))

        with open('LSTM_simu.txt', 'w') as f:
            f.write('Best Validation Epoch: {}\n'.format(ei))
            f.write('Best Validation Loss: {}\n'.format(best_valid_loss))
            f.write('Train Loss: {}\n'.format(train_loss))
            f.write('Test Loss: {}\n'.format(test_loss))

   
        torch.save(simulstm,'LSTM_simu.pt')
        
time_end=time.time()
print('Run Time (s):' ,time_end-time_start)
print('Run Time per Epoch (s):'(time_end-time_start)/n_epochs)

In [ ]:
# DSTM
X_spt=joblib.load('X_cov')
Z=joblib.load('Data_nonGaussian')
X_spt=X_spt.float().cuda()
Z=Z.float().cuda()
T=1095
S=50
Z_tmp=np.array(Z.cpu())
Z=[]
for i in Z_tmp[:,:50]:
    Z.append(np.c_[i])
X_cov=np.array(X_spt.cpu())[:50,:]
l=np.array([[1] for i in range(50)])
X_cov=np.concatenate((l,X_cov),axis=1)

In [ ]:
def sam_beta0(set_beta,set_omega,set_g):
    V0=inv((set_g[-1]**2)*inv(set_omega[-1])+np.identity(len(sample_beta[0][0])))
    a0=set_g[-1]*(inv(set_omega[-1])@set_beta[-1][0])+np.c_[[0.5 for i in range(11)]]
    new=rng.multivariate_normal(V0@a0.flatten(), V0, (1,),'raise',method='cholesky')
    return(new.T)

def sam_beta(t,set_beta0,set_beta,set_betaT,set_nu,set_omega,set_g):
    Vt=inv(X_cov.T@inv(set_nu[-1])@X_cov+(1+set_g[-1]**2)*inv(set_omega[-1]))
    if t==1:
        at=X_cov.T@inv(set_nu[-1])@Z[t-1]+set_g[-1]*inv(set_omega[-1])@set_beta0[-1]+set_g[-1]*inv(set_omega[-1])@set_beta[-2][t]
    elif t==T-1:
        at=X_cov.T@inv(set_nu[-1])@Z[t-1]+set_g[-1]*inv(set_omega[-1])@set_beta[-1][t-2]+set_g[-1]*inv(set_omega[-1])@set_betaT[-1]
    else:
        at=X_cov.T@inv(set_nu[-1])@Z[t-1]+set_g[-1]*inv(set_omega[-1])@set_beta[-1][t-2]+set_g[-1]*inv(set_omega[-1])@set_beta[-2][t]
    new=rng.multivariate_normal(Vt@at.flatten(), Vt, (1,),'raise',method='cholesky')
    return(new.T)
    
def sam_betaT(set_beta,set_nu,set_omega,set_g):
    VT=inv(X_cov.T@inv(set_nu[-1])@X_cov+inv(set_omega[-1]))
    aT=X_cov.T@inv(set_nu[-1])@Z[-1]+set_g[-1]*inv(set_omega[-1])@set_beta[-1][-1]
    new=rng.multivariate_normal(VT@aT.flatten(), VT, (1,),'raise',method='cholesky')
    return(new.T)

def sam_nu(set_beta,set_betaT,set_g):
    tmp=0
    fulldata=[(Z[i]-X_cov@set_beta[-1][i]) for i in range(T-1) if len(Z[i])==S]
    for i in fulldata:
        tmp=tmp+i@i.T
    tmp=tmp+(Z[-1]-X_cov@set_betaT[-1])@(Z[-1]-X_cov@set_betaT[-1]).T
    new=wishart.rvs(df=n_nu+len(fulldata)+1, scale=inv(tmp+V_nu), size=1, random_state=None)
    return(inv(new))
    
def sam_omega(set_beta0,set_beta,set_betaT,set_g):
    tmp=0
    fullcov=[(set_beta[-1][i]-set_g[-1]*set_beta[-1][i-1]) for i in range(1,T-1)]
    fullcov.append(set_beta[-1][0]-set_g[-1]*set_beta0[-1])
    fullcov.append(set_betaT[-1]-set_g[-1]*set_beta[-1][-1])
    for i in fullcov:
        tmp=tmp+i@i.T
    new=wishart.rvs(df=n_omega+T, scale=inv(tmp+V_omega), size=1, random_state=None)
    return(inv(new))

def sam_g(set_beta0,set_beta,set_betaT,set_omega):
    new=-1
    vg=np.sum([set_beta[-1][i].T@inv(set_omega[-1])@set_beta[-1][i] for i in range(0,T-1)])
    vg=vg+set_beta0[-1].T@inv(set_omega[-1])@set_beta0[-1]
    ag=np.sum([set_beta[-1][i+1].T@inv(set_omega[-1])@set_beta[-1][i] for i in range(0,T-2)])
    ag=ag+set_betaT[-1].T@inv(set_omega[-1])@set_beta[-1][-1]
    ag=ag+set_beta[-1][0].T@inv(set_omega[-1])@set_beta0[-1]
    while new<=0 or new>=1:
        new=rng.normal(loc=ag/vg,scale=np.sqrt(1/vg))
    return(new)

In [ ]:
#main Gibbs
T=1095
S=50
N_gibbs=40000
n_nu=S+10
n_omega=11+10
V_nu=np.identity(S)/(n_nu-S-1)
V_omega=np.identity(11)/(n_omega-12)
sample_beta0=[rng.multivariate_normal([0.5 for i in range(11)],np.identity(11),(1,),'raise',method='cholesky').T]
sample_betaT=[np.c_[rng.uniform(0.001,1,size=11)]]
sample_nu=[invwishart.rvs(df=n_nu, scale=(n_nu-S-1)*np.identity(S), size=1, random_state=None)]
sample_omega=[invwishart.rvs(df=n_omega, scale=(n_omega-12)*np.identity(11), size=1, random_state=None)]
sample_g=[rng.uniform(0.001,1)]
sample_beta=[[np.c_[rng.uniform(0.001,1,size=11)] for i in range(T-1)]]

time_start=time.time()
while(len(sample_beta0))<2:
    if len(sample_beta0)==1:
        print('Sampling Start')
    if len(sample_beta0)==10000:
        print('25% finished')
    elif len(sample_beta0)==20000:
        print('50% finished')
    elif len(sample_beta0)==30000:
        print('75% finished')
    sample_beta0.append(sam_beta0(sample_beta,sample_omega,sample_g))
    sample_beta.append([np.c_[rng.uniform(0.001,1,size=11)] for i in range(T-1)])
    for t in range(1,T):
        sample_beta[-1][t-1]=sam_beta(t,sample_beta0,sample_beta,sample_betaT,sample_nu,sample_omega,sample_g)
    sample_betaT.append(sam_betaT(sample_beta,sample_nu,sample_omega,sample_g))
    sample_nu.append(sam_nu(sample_beta,sample_betaT,sample_g))
    sample_omega.append(sam_omega(sample_beta0,sample_beta,sample_betaT,sample_g))
    sample_g.append(sam_g(sample_beta0,sample_beta,sample_betaT,sample_omega))
time_end=time.time()
joblib.dump(sample_beta0,'sample_beta0')
joblib.dump(sample_beta,'sample_beta')
joblib.dump(sample_betaT,'sample_betaT')
joblib.dump(sample_nu,'sample_nu')
joblib.dump(sample_omega,'sample_omega')
joblib.dump(sample_g,'sample_g')
print('Run Time:' ,time_end-time_start)

In [ ]:
beta0sample=[i[0][0] for i in sample_beta0[int(0.5*N_gibbs):]]
beta0burnin=[i[0][0] for i in sample_beta0[0:int(0.5*N_gibbs)]]
betasample=[i for i in sample_beta[int(0.5*N_gibbs):]]
betaburnin=[i for i in sample_beta[0:int(0.5*N_gibbs)]]
beta_avg=np.mean(betasample,axis=0)

betaTsample=[i for i in sample_betaT[int(0.5*N_gibbs):]]
betaTburnin=[i for i in sample_betaT[0:int(0.5*N_gibbs)]]
betaT_avg=np.mean(betaTsample,axis=0)
betaT_avg=betaT_avg.reshape(1,12,1)
beta_avg=np.concatenate((beta_avg,betaT_avg),axis=0)

gsample=[i[0][0] for i in sample_g[int(0.5*N_gibbs):]]
gburnin=[i[0][0] for i in sample_g[1:int(0.5*N_gibbs)]]
gburnin.insert(0,sample_g[0])
g_burnin_avg=[np.mean(gburnin[:i]) for i in range(1,len(beta0burnin)+1)]
g_sample_avg=[np.mean(gsample[:i]) for i in range(1,len(beta0sample)+1)]

nusample=[i for i in sample_nu[int(0.5*N_gibbs):]]
nuburnin=[i for i in sample_nu[0:int(0.5*N_gibbs)]]
nusample_avg=np.sum(nusample,axis=0)/len(nusample)

In [ ]:
# Compute MSE
Z_pred=[]
for t in range(1095):
    tmp=X_cov@beta_avg[t]+rng.multivariate_normal([0 for i in range(50)],nusample_avg,(1,),'raise',method='cholesky').T
    Z_pred.append(np.c_[tmp])
print('Training MSE:',sk.mean_squared_error(np.array(Z_pred).flatten(),np.array(Z).flatten()))

X_test=np.array(X_spt.cpu())[50:100,:]
X_test=np.concatenate((l,X_test),axis=1)
Z_test=[]
for i in Z_tmp[:,50:100]:
    Z_test.append(np.c_[i])
Z_pred2=[]

for t in range(1095):
    tmp=X_test@beta_avg[t]+rng.multivariate_normal([0 for i in range(50)],nusample_avg,(1,),'raise',method='cholesky').T
    Z_pred2.append(np.c_[tmp])
print('Test MSE:',sk.mean_squared_error(np.array(Z_pred2).flatten(),np.array(Z_test).flatten()))

In [ ]:
# Compute Regression Coefficient
for i in range(12):
    print('Covariate',i,':',np.mean(beta_avg,axis=0)[i])
print('------------')
for i in range(2,12):
    print('Covariate',i,':',np.mean(beta_avg,axis=0)[i]-np.mean(beta_avg,axis=0)[2])